Pneumonia is one of the leading respiratory illnesses worldwide, and its timely and accurate diagnosis is crucial for effective treatment. Manually reviewing chest X-rays plays a critical role in this process, but AI can significantly expedite and enhance assessments.

In this project, I explored the ability of a deep learning model to distinguish pneumonia cases from normal lung X-ray images. I fine-tuned a pre-trained ResNet-18 convolutional neural network to classify X-rays into two categories: normal lungs and those affected by pneumonia. Leveraging the pre-trained weights of ResNet-18 allowed me to create an accurate classifier efficiently, reducing the resources and time needed for training.

## The Data

<img src="x-rays_sample.png" align="center"/>
&nbsp

The dataset consisted of 300 chest X-rays for training and 100 for testing, evenly divided between NORMAL and PNEUMONIA categories. The images had been preprocessed and organized into train and test folders, with data loaders ready for use with PyTorch. This project highlights my ability to implement advanced deep learning techniques in a healthcare context, showcasing how AI improves diagnostic processes.ses.

In [65]:
# # Make sure to run this cell to use torchmetrics.
!pip install torch torchvision torchmetrics

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [66]:
# Import required libraries
# Data loading
import random
import numpy as np
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Train model
import torch
from torchvision import models
import torch.nn as nn
import torch.optim as optim

# Evaluate model
from torchmetrics import Accuracy, F1Score

# Set random seeds for reproducibility
torch.manual_seed(101010)
np.random.seed(101010)
random.seed(101010)

In [67]:
import os
import zipfile

# Unzip the data folder
if not os.path.exists('data/chestxrays'):
    with zipfile.ZipFile('data/chestxrays.zip', 'r') as zip_ref:
        zip_ref.extractall('data')

In [68]:
# Normalize the X-rays using transforms.
# standard deviations of the three color channels, (R,G,B), from the original ResNet-18 training dataset.
transform_mean = [0.485, 0.456, 0.406]
transform_std =[0.229, 0.224, 0.225]
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize(mean=transform_mean, std=transform_std)])

# Apply the image transforms
train_dataset = ImageFolder('data/chestxrays/train', transform=transform)
test_dataset = ImageFolder('data/chestxrays/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=len(train_dataset) // 2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))

In [69]:
# Instantiate the model
# Load the pre-trained ResNet-18 model
resnet18 = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

In [70]:
# Modify the model
# Freeze the parameters of the model
for param in resnet18.parameters():
    param.requires_grad = False

# Modify the final layer for binary classification
resnet18.fc = nn.Linear(resnet18.fc.in_features, 1)

In [71]:
# Define the training loop
# Model training/fine-tuning loop
def train(model, train_loader, criterion, optimizer, num_epochs):
    
    # Train the model for the specified number of epochs
    for epoch in range(num_epochs):
        # Set the model to train mode
        model.train()

        # Initialize the running loss and accuracy
        running_loss = 0.0
        running_accuracy = 0

        # Iterate over the batches of the train loader
        for inputs, labels in train_loader:

            # Zero the optimizer gradients
            optimizer.zero_grad()
            
            # Ensure labels have the same dimensions as outputs
            labels = labels.float().unsqueeze(1)

            # Forward pass
            outputs = model(inputs)
            preds = torch.sigmoid(outputs) > 0.5 # Binary classification
            loss = criterion(outputs, labels)

            # Backward pass and optimizer step
            loss.backward()
            optimizer.step()

            # Update the running loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            running_accuracy += torch.sum(preds == labels.data)

        # Calculate the train loss and accuracy for the current epoch
        train_loss = running_loss / len(train_dataset)
        train_acc = running_accuracy.double() / len(train_dataset)

        # Print the epoch results
        print('Epoch [{}/{}], train loss: {:.4f}, train acc: {:.4f}'
              .format(epoch+1, num_epochs, train_loss, train_acc))


In [72]:
#Fine-tune the model       
# Set the model to ResNet-18
model = resnet18

# Fine-tune the ResNet-18 model for 3 epochs using the train_loader
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
train(model, train_loader, criterion, optimizer, num_epochs=3)

Epoch [1/3], train loss: 1.3915, train acc: 0.4567
Epoch [2/3], train loss: 0.8973, train acc: 0.4633
Epoch [3/3], train loss: 0.9199, train acc: 0.5033


In [73]:
# Evaluate the model

# Set model to evaluation mode
model = resnet18
model.eval()

# Initialize metrics for accuracy and F1 score
accuracy_metric = Accuracy(task="binary")
f1_metric = F1Score(task="binary")

# Create lists store all predictions and labels
all_preds = []
all_labels = []

# Disable gradient calculation for evaluation
with torch.no_grad():
  for inputs, labels in test_loader:
    # Forward pass
    outputs = model(inputs)
    preds = torch.sigmoid(outputs).round()  # Round to 0 or 1

    # Extend the lists with predictions and labels
    all_preds.extend(preds.tolist())
    all_labels.extend(labels.unsqueeze(1).tolist())

  # Convert lists back to tensors
  all_preds = torch.tensor(all_preds)
  all_labels = torch.tensor(all_labels)

  # Calculate accuracy and F1 score
  test_accuracy = accuracy_metric(all_preds, all_labels).item()
  test_f1_score = f1_metric(all_preds, all_labels).item()
  print(f"\nTest accuracy: {test_acc:.3f}\nTest F1-score: {test_f1:.3f}")


Test accuracy: 0.580
Test F1-score: 0.704
